In [1]:
import torch
import os
import pandas as pd

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from module.model import model as tagger
from module.model import device
from module.model import tokenizer
from module.model import id2label

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


36223


## Loading

In [3]:
file_list = os.listdir('./inference_data')
file_path = []
for file_name in file_list:
    tmp_path = os.path.join('./inference_data', file_name)
    file_path.append(tmp_path)
file_path.sort()

In [4]:
load_df = lambda x: pd.read_csv(x)
dfs = [load_df(x) for x in file_path]
infer_data = dfs[0]
infer_data.dropna(inplace=True)
# infer_data[pd.isna(infer_data).any(axis=1)]
# infer_data.isna().nunique()
print(f'length of {file_path[0]}: {len(infer_data)}')

length of ./inference_data/event_port.csv: 171


In [5]:
infer_data.isnull().any()

Event    False
dtype: bool

## Cleansing

In [6]:
# def preprocess(df):
#     df['Event'] = df['Event'].str.replace("．", ".", regex=False)
#     df['Event'] = df['Event'].astype(str)
#     # df['Event'] = df['Event'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', " ", regex=True)
#     return df

In [7]:
# infer_data = preprocess(infer_data)

In [8]:
infer_data_lst = infer_data.Event.apply(lambda x: tokenizer(x, max_length=256, padding='max_length', truncation=True))
input_ids_lst = []
input_masks_lst = []
for e in infer_data_lst:
    input_ids_lst.append(e['input_ids'])
    input_masks_lst.append(e['attention_mask'])
len(input_ids_lst), len(input_masks_lst)

(171, 171)

In [9]:
def infer(input_ids_lst, input_masks_lst):

    tagger.eval()
    with torch.no_grad():
        infer_results = []
        for input_ids, input_masks in zip(input_ids_lst, input_masks_lst): 
            input_tensor = torch.LongTensor(input_ids).unsqueeze(0).to(device)
            mask_tensor = torch.LongTensor(input_masks).unsqueeze(0).to(device)
            output = tagger(input_tensor, mask_tensor)
            pred = torch.argmax(output, dim=-1).squeeze().detach().cpu().tolist()
            infer_results.append(pred)

        return infer_results
    
infer_results = infer(input_ids_lst, input_masks_lst)

In [10]:
subwordsList = []
tagsList = []
toLabel = lambda x: id2label[x]
for input_ids, result in zip(input_ids_lst, infer_results):
    decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
    subwords = tokenizer.encode(decoded)[1:-1]
    subwords = list(map(lambda x: tokenizer.convert_ids_to_tokens(x), subwords))
    subwordsList.append(subwords)
    lenSubwords = len(subwords)    

    result = result[1:lenSubwords+1]
    tags = []
    for id in result:
        tags.append(toLabel(id))
    tagsList.append(tags)

In [11]:
# 날짜(DAT), 시간(TIME), 장소(LOC), 작업 정보(WRK)를 추출해서 txt 파일에 저장

def extract(tagtype, tags, subwords):
    # if tag does not exists then notify and pass
    # if not '{tag}_I' then stop
    tag_start = f'{tagtype}_B'
    if tag_start in tags:
        b_idx = tags.index(tag_start)
        for i, tag in enumerate(tags[b_idx:]):
            if i > 0 and tag != f'{tagtype}_I':
                # print(tags[b_idx:b_idx+i+1])
                # print(tags[b_idx:b_idx+i])
                tagged = subwords[b_idx:b_idx+i]
                tagged = tokenizer.convert_tokens_to_string(tagged)
                print(tagged)
                return tagged
    else:
        msg = f'{tagtype} not found.'
        print(msg)
        return msg

tagtypes = dict(
    dat='DAT',
    tim='TIM',
    loc='LOC',
    wrk='WRK',    
)

# subwordsList, tagsList
nerResults = []
count = 0
for subwords, tags in zip(subwordsList, tagsList):
    print(f'index: {count}')
    extracted = []
    for tagtype in tagtypes.values():
        print(tagtype)
        tagged = extract(tagtype, tags, subwords)
        extracted.append(tagged)
        print()
    nerResults.append(extracted)
    count += 1

index: 0
DAT
2015년 3월 15일

TIM
20 : 35경

LOC
○○ 에 선적을

WRK
WRK not found.

index: 1
DAT
2013년 2월 일 11

TIM
: 30경 부

LOC
○○

WRK
수행하던 위에서 를 하

index: 2
DAT
2021 . 6 . . ( 금 )

TIM
: 5분경 , 서구 소재

LOC
00 에서 25ton

WRK
에 톤 를 걸어주고

index: 3
DAT
2016년 7월

TIM
TIM not found.

LOC
울산 에서 에

WRK
##기 위해 쪽

index: 4
DAT
2017년 2월

TIM
TIM not found.

LOC
울산 에서 을

WRK
위해 육상

index: 5
DAT
2018년 2월

TIM
TIM not found.

LOC
LOC not found.

WRK
엔지 에서 에 용

index: 6
DAT
2016년 3월

TIM
TIM not found.

LOC
울산 에서 이

WRK
WRK not found.

index: 7
DAT
2013년 월 1일 항

TIM
TIM not found.

LOC
1 18 에서 로 2m 에서

WRK
가 떨어져 목과

index: 8
DAT
2018년 5월

TIM
TIM not found.

LOC
울산 에서 코일

WRK
을 위해 2번

index: 9
DAT
2015년 3월 18일

TIM
TIM not found.

LOC
항에서 원당

WRK
에 떨어져 원당

index: 10
DAT
201 년 6월 5일 항

TIM
TIM not found.

LOC
1 18 에서 로 사이로 하여

WRK
머리와 다리가

index: 11
DAT
2012년 11월 28일

TIM
TIM not found.

LOC
항 1 에서 화물 상

WRK
· 하차 에 연결된 를

index: 12
DAT
2017년 2월 6일 ( 월 )

TIM
TIM not found.

LOC
○○항에서

WRK
벌크 상부 해치 (

index: 13

In [12]:
for i in nerResults:
    print(i)

['2015년 3월 15일', '20 : 35경', '○○ 에 선적을', 'WRK not found.']
['2013년 2월 일 11', ': 30경 부', '○○', '수행하던 위에서 를 하']
['2021 . 6 . . ( 금 )', ': 5분경 , 서구 소재', '00 에서 25ton', '에 톤 를 걸어주고']
['2016년 7월', 'TIM not found.', '울산 에서 에', '##기 위해 쪽']
['2017년 2월', 'TIM not found.', '울산 에서 을', '위해 육상']
['2018년 2월', 'TIM not found.', 'LOC not found.', '엔지 에서 에 용']
['2016년 3월', 'TIM not found.', '울산 에서 이', 'WRK not found.']
['2013년 월 1일 항', 'TIM not found.', '1 18 에서 로 2m 에서', '가 떨어져 목과']
['2018년 5월', 'TIM not found.', '울산 에서 코일', '을 위해 2번']
['2015년 3월 18일', 'TIM not found.', '항에서 원당', '에 떨어져 원당']
['201 년 6월 5일 항', 'TIM not found.', '1 18 에서 로 사이로 하여', '머리와 다리가']
['2012년 11월 28일', 'TIM not found.', '항 1 에서 화물 상', '· 하차 에 연결된 를']
['2017년 2월 6일 ( 월 )', 'TIM not found.', '○○항에서', '벌크 상부 해치 (']
['DAT not found.', 'TIM not found.', 'LOC not found.', '산화물 ( 광물 ) 을']
['DAT not found.', 'TIM not found.', '있던 가', '기 을 데크']
['DAT not found.', 'TIM not found.', 'LOC not found.', '##물의 과 0 포대물에 의해 무게 심']
['2012 . 6 . 7

In [16]:
df = pd.DataFrame(nerResults, columns=['Date', 'Time', 'Location', 'Work'])

In [18]:
df.to_csv('./inference_result.txt', index=False, sep='/')